In [9]:
import semantic_kernel as sk

kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

api_key, org_id = sk.openai_settings_from_dot_env()
service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-0125", api_key=api_key, org_id=org_id),
)

plugin = kernel.import_plugin_from_prompt_directory("./Plugins/", "ReportPlugin")
OutlineJSON_function = plugin["OutlineJSON"]
OutlineJSON = await kernel.invoke(OutlineJSON_function, sk.KernelArguments(input="Báo cáo thị trường Tiêu dùng năm 2023", lang = "Vietnamese", critics = ""))

print(OutlineJSON)

critics_function = plugin["Critics"]
critics = await kernel.invoke(critics_function, sk.KernelArguments(input = "OutlineJSON", prompt = OutlineJSON_function.prompt_template.prompt_template_config.template))
print(critics)

OutlineJSON = await kernel.invoke(OutlineJSON_function, sk.KernelArguments(input="Báo cáo thị trường Tiêu dùng năm 2023", lang = "Vietnamese", critics = critics))

print(OutlineJSON)

{'report': {'title': 'Báo cáo thị trường Tiêu dùng năm 2023','introduction': 'Báo cáo này sẽ cung cấp cái nhìn tổng quan về thị trường tiêu dùng trong năm 2023, bao gồm xu hướng, dự đoán và các yếu tố ảnh hưởng.','body': {'headings': ['Tổng quan về thị trường tiêu dùng','Xu hướng tiêu dùng mới','Dự đoán về sự phát triển của thị trường','Yếu tố kinh tế và xã hội ảnh hưởng đến thị trường','Thách thức và cơ hội trong ngành','Ảnh hưởng của công nghệ vào thị trường tiêu dùng','Chiến lược tiếp thị và quảng cáo trong năm 2023']},'conclusion': 'Tóm lại, thị trường tiêu dùng năm 2023 đang đối diện với nhiều thách thức nhưng cũng mang đến nhiều cơ hội cho các doanh nghiệp.','recommendations': 'Dựa vào những thông tin trong báo cáo, chúng tôi đề xuất các doanh nghiệp nên tập trung vào nâng cao chất lượng sản phẩm, áp dụng công nghệ mới và tối ưu hóa chiến lược tiếp thị để cạnh tranh hiệu quả trên thị trường.'}}


FunctionResult(function=KernelFunctionMetadata(name='Critics', plugin_name='ReportPlugin', description='Write a critic for current tasks', parameters=[KernelParameterMetadata(name='input', description="The previous task's output.", default_value='', type_='', is_required=True, type_object=None), KernelParameterMetadata(name='prompt', description="The previous task's prompt", default_value='', type_='', is_required=True, type_object=None)], is_prompt=True, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='The completion result', default_value=None, type_='FunctionResult', is_required=True, type_object=None)), value=[OpenAIChatMessageContent(inner_content=ChatCompletion(id='chatcmpl-9D18FCBgYEqPUUuvgUQtclSHabJE4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='- Provide a more detailed outline with specific headings for each section (Introduction, Body, Conclusion, Recommendations)\n- Ensure th

In [10]:
print(critics)

- Provide a more detailed outline with specific headings for each section (Introduction, Body, Conclusion, Recommendations)
- Ensure that each section is clearly defined and includes relevant content
- Include a descriptive title for the report
- Make sure the outline is well-structured and easy to follow for the reader
- Aim for 6-8 headings in the body section to cover various aspects of the topic
- Provide specific and actionable recommendations in the recommendations section
- Ensure that the introduction sets the stage for the report and the conclusion summarizes key points effectively


In [2]:
from Plugins.utils import json_split
outline_dict = json_split(str(OutlineJSON))
outline_dict

{'title': 'Báo cáo thị trường Tiêu dùng năm 2023',
 'introduction': 'Báo cáo này sẽ cung cấp cái nhìn tổng quan về thị trường tiêu dùng trong năm 2023, bao gồm xu hướng, dự đoán và các yếu tố ảnh hưởng.',
 'body': {'headings': ['Tổng quan về thị trường tiêu dùng',
   'Xu hướng tiêu dùng mới',
   'Dự đoán về sự phát triển của thị trường',
   'Yếu tố ảnh hưởng đến thị trường',
   'Thị trường tiêu dùng trong bối cảnh kinh tế hiện nay',
   'Ảnh hưởng của công nghệ vào thị trường tiêu dùng']},
 'conclusion': 'Tóm lại, thị trường tiêu dùng năm 2023 đang phát triển mạnh mẽ và có nhiều cơ hội cho các doanh nghiệp.',
 'recommendations': 'Dựa vào những thông tin trong báo cáo, chúng tôi khuyến nghị các doanh nghiệp nên tập trung vào nghiên cứu thị trường và phát triển sản phẩm dựa trên nhu cầu của người tiêu dùng để tận dụng cơ hội thị trường.'}

In [3]:
Introduction_function = plugin["Introduction"]
Introduction = await kernel.invoke(Introduction_function, sk.KernelArguments(introduction=outline_dict["introduction"]
                                                                            , reportLayout = str(OutlineJSON)))
print(Introduction)

Báo cáo thị trường Tiêu dùng năm 2023 sẽ cung cấp cho bạn cái nhìn tổng quan về thị trường tiêu dùng trong năm 2023, bao gồm các xu hướng mới, dự đoán về sự phát triển của thị trường, các yếu tố ảnh hưởng đến thị trường, cũng như sự phân chia giữa thị trường tiêu dùng trực tuyến và truyền thống. Với sự phát triển mạnh mẽ, thị trường tiêu dùng năm 2023 mang đến nhiều cơ hội cho các doanh nghiệp.


In [4]:
from Plugins.google_search import GoogleSearchPlugin
from Plugins.utils import token_count
import asyncio
TOKEN_LIMIT = 10000

google_plugin = kernel.import_plugin_from_object(GoogleSearchPlugin(), plugin_name = "GoogleSearchPlugin")
google_search_function = google_plugin["GoogleSearch"]
get_content_function = google_plugin["GetContentFromURL"]

column_function = plugin["Column"]
column_summ_function = plugin["ColumnSumm"]
headings = outline_dict["body"]["headings"]
query_function = plugin["Query"]

async def columnist(heading, title):
    search_result = []
    tokens = 0
    query = str(await kernel.invoke(query_function, sk.KernelArguments(input = heading, big_title = title)))
    search_result_urls = await kernel.invoke(google_search_function, sk.KernelArguments(keyword = query, num_results = 15))
    search_result_urls = search_result_urls.value
    print(search_result_urls)
    while True:
        if not search_result_urls:
            break
        url = search_result_urls.pop(0)
        content = await kernel.invoke(get_content_function, sk.KernelArguments(url = url))
        content = content.value
        new_tokens = token_count(content)
        if tokens + new_tokens > TOKEN_LIMIT:
            print(tokens)
            if tokens >= TOKEN_LIMIT//2:
                break
            else:
                continue
        tokens += new_tokens
        search_result.append(content)
    search_result = " ".join(search_result)
    column = await kernel.invoke(column_function, sk.KernelArguments(input = search_result, body_content = heading))
    column_summ = await kernel.invoke(column_summ_function, sk.KernelArguments(input = search_result, body_content = heading))
    return str(column), str(column_summ)

coros = []
for heading in headings:
    coros.append(columnist(heading, outline_dict["title"]))

values = await asyncio.gather(*coros)
print(values)




['https://www.gso.gov.vn/du-lieu-va-so-lieu-thong-ke/2024/04/thi-truong-gia-quy-i-2024-va-du-bao-yeu-to-tac-dong-cpi-nam-2024/', 'https://baodautu.vn/4-yeu-to-anh-huong-den-thi-truong-hang-tieu-dung-nhanh-viet-nam-trong-2023-d185537.html', 'https://www.mof.gov.vn/webcenter/portal/ttpltc/pages_r/l/chi-tiet-tin-ttpltc?dDocName=MOFUCM273389', 'https://www.cimigo.com/vi/trends/xu-huong-tieu-dung-viet-nam-2023/', 'https://special.nhandan.vn/kinh-te-Vietnam-nam-2023-mot-nam-vuot-kho-thach-thuc-van-con/index.html', 'https://vneconomy.vn/xu-huong-tieu-dung-hien-dai-thau-hieu-de-chinh-phuc-thi-truong.htm', 'https://aglobal.vn/blog/xu-huong-tieu-dung-quy-4-nam-2023-nhung-thay-doi-can-luu-y-1564161623', 'https://mof.gov.vn/webcenter/portal/vclvcstc/pages_r/l/chi-tiet-tin?dDocName=MOFUCM262964', 'https://www.gso.gov.vn/du-lieu-va-so-lieu-thong-ke/2024/01/cac-yeu-to-tac-dong-den-lam-phat-nam-2023-va-mot-so-giai-phap-kiem-soat-lam-phat-trong-nam-2024/', 'https://pmax.com.vn/blog/xu-huong-thi-truong-

In [5]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])
    print(token_count(value[1]), "tokens")
    print(value[1])

747 tokens
Trên thị trường tiêu dùng Việt Nam, năm 2023, có một số xu hướng quan trọng đang diễn ra và ảnh hưởng đến sự phát triển kinh tế của đất nước. Đầu tiên, cơ cấu dân số của Việt Nam đang đóng vai trò quan trọng trong việc thúc đẩy tăng trưởng kinh tế. Với tỷ lệ tham gia lao động cao, đặc biệt là phụ nữ, Việt Nam có một lực lượng lao động lớn và rẻ. Điều này giúp giảm tỷ lệ phụ thuộc và tạo điều kiện cho GDP tăng trưởng mạnh mẽ.

Ngoài ra, tăng trưởng GDP bình quân 5.9% trong mười năm qua cũng là một yếu tố quan trọng khác. Việt Nam đã thu hút đầu tư nước ngoài và tăng cường sản xuất, đóng góp vào sự phát triển của nền kinh tế. Sự tăng trưởng trong ngành sản xuất và tiêu thụ hàng hóa và dịch vụ bán lẻ cũng đang tạo đà phát triển tích cực.

Du lịch nội địa cũng đang trở thành một nguồn thu hút lớn, lấp đầy khoảng trống từ du lịch quốc tế. Việt Nam đang thúc đẩy du lịch trong nước và tạo điều kiện cho ngành du lịch phát triển bền vững. Nền kinh tế Internet cũng đang phát triển mạn

In [6]:
conclusion_function = plugin["Conclusion"]
summContent = " ".join([value[1] for value in values])
conclusion = await kernel.invoke(conclusion_function, sk.KernelArguments(conclusion = outline_dict["conclusion"], summContent = summContent))
print(str(conclusion))


Tổng kết lại, thị trường tiêu dùng tại Việt Nam trong năm 2023 đang trải qua một giai đoạn phát triển tích cực và đầy tiềm năng. Với sự tăng trưởng ổn định của nền kinh tế, cơ cấu dân số thuận lợi, sự phát triển của các ngành công nghiệp và xu hướng tiêu dùng mới, Việt Nam đang chứng tỏ sức hút và tiềm năng phát triển trong thời gian tới.

Các yếu tố như tăng trưởng GDP, sự gia tăng của tầng lớp trung lưu, sự phát triển của du lịch trong nước và sự ổn định về chính trị đều đóng vai trò quan trọng trong việc thúc đẩy sự phát triển của thị trường tiêu dùng tại Việt Nam. Đồng thời, việc nắm bắt và tận dụng các xu hướng tiêu dùng mới như mua sắm trực tuyến, sức khỏe và lối sống lành mạnh cũng sẽ giúp các doanh nghiệp tạo ra giá trị và cạnh tranh hiệu quả trên thị trường.

Với những cơ hội và thách thức đặt ra, các doanh nghiệp cần áp dụng các chiến lược linh hoạt, sáng tạo để thích ứng và phát triển trong môi trường kinh doanh ngày càng cạnh tranh. Việc hiểu rõ về nhu cầu và mong muốn của 

In [7]:
print(token_count(str(conclusion)))

666


In [8]:
recommendation_function = plugin["Recommendation"]
summContent = " ".join([value[1] for value in values])
recommendation = await kernel.invoke(recommendation_function, sk.KernelArguments(recommendation = outline_dict["recommendations"], summContent = summContent))
print(str(recommendation))
print(token_count(str(recommendation)))

Dựa vào những thông tin trong báo cáo, chúng tôi đề xuất các doanh nghiệp nên tập trung vào việc nâng cao chất lượng sản phẩm/dịch vụ, tận dụng thị trường trực tuyến và xây dựng chiến lược tiếp thị hiệu quả. Việc nâng cao chất lượng sản phẩm/dịch vụ sẽ giúp doanh nghiệp thu hút và giữ chân khách hàng, tạo lòng tin và uy tín trong mắt người tiêu dùng. Đồng thời, việc tận dụng thị trường trực tuyến sẽ giúp doanh nghiệp tiếp cận đến đông đảo khách hàng tiềm năng, mở rộng phạm vi kinh doanh và tạo ra cơ hội tăng trưởng. Xây dựng chiến lược tiếp thị hiệu quả sẽ giúp doanh nghiệp tiếp cận đúng đối tượng khách hàng, tạo ấn tượng mạnh mẽ và thu hút sự chú ý trong môi trường cạnh tranh gay gắt hiện nay. Việc thực hiện ba yếu tố này đồng thời sẽ giúp doanh nghiệp đạt được hiệu quả kinh doanh cao nhất và phát triển bền vững trên thị trường tiêu dùng tại Việt Nam trong năm 2023.
416


In [9]:
for value in values:
    print(token_count(value[0]), "tokens")
    print(value[0])

747 tokens
Trên thị trường tiêu dùng Việt Nam, năm 2023, có một số xu hướng quan trọng đang diễn ra và ảnh hưởng đến sự phát triển kinh tế của đất nước. Đầu tiên, cơ cấu dân số của Việt Nam đang đóng vai trò quan trọng trong việc thúc đẩy tăng trưởng kinh tế. Với tỷ lệ tham gia lao động cao, đặc biệt là phụ nữ, Việt Nam có một lực lượng lao động lớn và rẻ. Điều này giúp giảm tỷ lệ phụ thuộc và tạo điều kiện cho GDP tăng trưởng mạnh mẽ.

Ngoài ra, tăng trưởng GDP bình quân 5.9% trong mười năm qua cũng là một yếu tố quan trọng khác. Việt Nam đã thu hút đầu tư nước ngoài và tăng cường sản xuất, đóng góp vào sự phát triển của nền kinh tế. Sự tăng trưởng trong ngành sản xuất và tiêu thụ hàng hóa và dịch vụ bán lẻ cũng đang tạo đà phát triển tích cực.

Du lịch nội địa cũng đang trở thành một nguồn thu hút lớn, lấp đầy khoảng trống từ du lịch quốc tế. Việt Nam đang thúc đẩy du lịch trong nước và tạo điều kiện cho ngành du lịch phát triển bền vững. Nền kinh tế Internet cũng đang phát triển mạn

In [6]:
import semantic_kernel as sk
from Plugins.google_search import GoogleSearchPlugin

kernel = sk.Kernel()

from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

api_key, org_id = sk.openai_settings_from_dot_env()
service_id = "default"
kernel.add_service(
    OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-0125", api_key=api_key, org_id=org_id),
)

plugin = kernel.import_plugin_from_prompt_directory("./Plugins/", "ReportPlugin")
Query_function = plugin["Query"]
Query = await kernel.invoke(Query_function, sk.KernelArguments(input="Xu hướng hợp tác quốc tế", big_title = "Xu hướng Anime 2023", lang = "Vietnamese"))
print(Query)

google_plugin = kernel.import_plugin_from_object(GoogleSearchPlugin(), plugin_name = "GoogleSearchPlugin")
google_search_function = google_plugin["GoogleSearch"]

print(await kernel.invoke(google_search_function, sk.KernelArguments(
                                                          keyword = Query, 
                                                          num_results = 10,
                                                          
                                                          )
                                                    ))

Variable `Symbols.VAR_PREFIX: critics` not found in the KernelArguments


"Xu hướng hợp tác quốc tế trong ngành công nghiệp Anime 2023" site:.vn
An error occurred: HTTP Error 429: Too Many Requests



In [2]:
print("{{}}")

{{}}
